In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (88 SW 7th St #4201, Miami, FL 33130, USA, (25...
2     (5500 Island Estates Dr, Aventura, FL 33160, U...
3     (1425 Brickell Ave #58e, Miami, FL 33131, USA,...
4     (2423 Fisher Island Dr #5203, Miami Beach, FL ...
5     (400 Alton Rd APT 2506, Miami Beach, FL 33139,...
6     (1100 Biscayne Blvd UNIT 3507, Miami, FL 33132...
7     (5500 Island Estates Dr #1106, Aventura, FL 33...
8     (1000 South Pointe Dr APT 101, Miami Beach, FL...
9     (3737 Collins Ave s 1103, Miami Beach, FL 3314...
10    (2 Grove Isle Dr B1702, Miami, FL 33133, USA, ...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Rise Condo\r88 SW 7th St 4201\rMiami,3/17/2022,5/11/2022,21,"$6,300,000.00",4424,"$1,424.05",Ana Maria Gomez,Fortune Intl Realty,TEMP USER,Miami Assoc of Realtors,Rise Condo 88 SW 7th St 4201 Miami,Rise Condo,88 SW 7th St 4201 Miami,"(88 SW 7th St #4201, Miami, FL 33130, USA, (25...","(25.7671967, -80.1945134, 0.0)",25.767197,-80.194513,0.0
2,Prive Condo\r5500 Island Estates Dr PH7\rAventura,8/17/2020,5/12/2022,561,"$5,800,000.00",4417,"$1,313.11",Bento Queiroz,"Compass Florida, LLC",Diana Shay,Miami New Home Realty,Prive Condo 5500 Island Estates Dr PH7 Aventura,Prive Condo,5500 Island Estates Dr PH7 Aventura,"(5500 Island Estates Dr, Aventura, FL 33160, U...","(25.946531, -80.130636, 0.0)",25.946531,-80.130636,0.0
3,Four Seasons Residences\r1425 Brickell Ave 58E...,2/3/2022,5/11/2022,34,"$4,061,000.00",3357,"$1,209.71",Carlos Herman Morean,"Compass Florida, LLC",Michael Light,Douglas Elliman,Four Seasons Residences 1425 Brickell Ave 58E ...,Four Seasons Residences,1425 Brickell Ave 58E Miami,"(1425 Brickell Ave #58e, Miami, FL 33131, USA,...","(25.7591323, -80.1918295, 0.0)",25.759132,-80.191829,0.0
4,Bayside Village Condo\r2423 Fisher Island Dr 5...,2/14/2022,5/10/2022,6,"$3,550,000.00",2234,"$1,589.08",Karla Abaunza,Luxury Living Realty,Karla Abaunza,Luxury Living Realty,Bayside Village Condo 2423 Fisher Island Dr 52...,Bayside Village Condo,2423 Fisher Island Dr 5203 Miami Beach,"(2423 Fisher Island Dr #5203, Miami Beach, FL ...","(25.7589251, -80.143131, 0.0)",25.758925,-80.143131,0.0
5,Murano Grande\r400 Alton Rd 2506\rMiami Beach,1/14/2022,5/11/2022,96,"$3,500,000.00",2024,"$1,729.25",Jeffrey Miller,One Sotheby's Intl Realty,Jonathan Corso,Coldwell Banker,Murano Grande 400 Alton Rd 2506 Miami Beach,Murano Grande,400 Alton Rd 2506 Miami Beach,"(400 Alton Rd APT 2506, Miami Beach, FL 33139,...","(25.7727159, -80.14041569999999, 0.0)",25.772716,-80.140416,0.0
6,Marquis\r1100 E Biscayne Blvd 3507\rMiami,11/18/2021,5/11/2022,158,"$3,200,000.00",3776,$847.46,Domenico Albano,A & D Group Realty LLC,Ozgur Ureten,NB Elite Realty,Marquis 1100 E Biscayne Blvd 3507 Miami,Marquis,1100 E Biscayne Blvd 3507 Miami,"(1100 Biscayne Blvd UNIT 3507, Miami, FL 33132...","(25.7852071, -80.1901214, 0.0)",25.785207,-80.190121,0.0
7,Prive Condo\r5500 Island Estates Dr 1106\rAven...,2/19/2020,5/12/2022,775,"$3,200,000.00",2877,"$1,112.27",Jonathan Lief,Aventura Real Estate,Nicole Holtzheuser,Florida Luxurious Properties,Prive Condo 5500 Island Estates Dr 1106 Aventura,Prive Condo,5500 Island Estates Dr 1106 Aventura,"(5500 Island Estates Dr #1106, Aventura, FL 33...","(25.946531, -80.130636, 0.0)",25.946531,-80.130636,0.0
8,Murano at Portofino Condo\r1000 S Pointe Dr 10...,11/18/2021,5/11/2022,110,"$3,200,000.00",1445,"$2,214.53",Carolina Rainer,Douglas Elliman,James Kay,Premier Brokers Int'l Inc,Murano at Portofino Condo 1000 S Pointe Dr 101...,Murano at Portofino Condo,1000 S Pointe Dr 101 Miami Beach,"(1000 South Pointe Dr APT 101, Miami Beach, FL...","(25.7690044, -80.1381107, 0.0)",25.769004,-80.138111,0.0
9,The Caribbean Condo\r3737 Collins Ave S-1103\r...,1/24/2022,5/10/2022,33,"$2,625,000.00",1621,"$1,619.37",Ida Schwartz,Compass Florida LLC,Stacy Robins,Stacy Robins Companies Inc,The Caribbean Condo 3737 Collins Ave S-1103 Mi...,The Caribbean Condo,3737 Collins Ave S-1103 Miami Beach,"(3737 Collins Ave s 1103, Miami Beach, FL 3314...","(25.810647, -80.122714, 0.0)",25.810647,-80.122714,0.0
10,G0rove Isle Condo\r2 Grove Isle Dr B1702\rMiami,3/28/2022,5/9/2022,11,"$2,070,000.00",2398,$863.22,Alec Macias,Sea Grove Realty,Alec Macias,Sea Grove Realty,G0rove Isle Condo 2 Grove Isle Dr B1702 Miami,G,0rove Isle Condo 2 Grove Isle Dr B1702 Miami,"(2 Grove Isle Dr B1702, Miami, FL 33133, USA, ...","(25.7351295, -80.21874609999999, 0.0)",25.73

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [13]:
df.at[10,'address_only']=('2 Grove Isle Dr B1702')
df.at[10,'building_name']=('Grove Isle Condo')

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [14]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('May 8th - May 14th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [15]:
m.save('index.html')